# Triage Issue

* This is a helper notebook to go with the triage module
* It can be used to triage issues.
* Issues needing Triage are added to [Needs Triage Project](https://github.com/orgs/kubeflow/projects/26)

In [15]:
import importlib
import sys
import os
os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path = [os.path.abspath(os.path.join(os.getcwd(), ".."))] + sys.path

In [17]:
import code_intelligence
from code_intelligence import graphql

In [18]:
client = graphql.GraphQLClient()

## Fetch Card Id

* This is a snippet 

In [24]:
project_query="""query projectCards($org: String!, $project: String!) {
  organization(login: $org) {
    projects(last: 1, search: $project) {
      totalCount
      edges {
        node {
          name
          url
          columns(first: 20) {
            totalCount
            pageInfo {
              endCursor
              hasNextPage
            }
            nodes {
              name
              id
            }
          }
        }
      }
    }
  }
}

"""
variables = {
    "org": "kubeflow",
    "project": "Bug Triage",
}
results = client.run_query(project_query, variables)
results

{'data': {'organization': {'projects': {'totalCount': 1,
    'edges': [{'node': {'name': 'Bug Triage',
       'url': 'https://github.com/orgs/kubeflow/projects/26',
       'columns': {'totalCount': 4,
        'pageInfo': {'endCursor': 'Y3Vyc29yOnYyOpIDzgBajOw=',
         'hasNextPage': False},
        'nodes': [{'name': 'Needs triage',
          'id': 'MDEzOlByb2plY3RDb2x1bW41OTM0MzEz'},
         {'name': 'High priority', 'id': 'MDEzOlByb2plY3RDb2x1bW41OTM0MzE0'},
         {'name': 'Low priority', 'id': 'MDEzOlByb2plY3RDb2x1bW41OTM0MzE1'},
         {'name': 'Closed', 'id': 'MDEzOlByb2plY3RDb2x1bW41OTM0MzE2'}]}}}]}}}}